In [2]:
import pandas as pd

df = pd.read_csv('/kaggle/input/daily-aqi-malang/air-quality-malang.csv')
df

,date,pm25
0,2025/8/1,54
1,2025/8/3,33
2,2025/8/4,79
3,2025/8/5,74
4,2025/8/6,82
...,...,...
1269,2021/12/29,29
1270,2021/12/30,44
1271,2021/12/31,49
1272,2022/1/1,54


In [3]:
df_cleaned = df.copy()
df_cleaned = df_cleaned.rename(columns=lambda x: x.strip())
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'], format='%Y/%m/%d', errors='coerce')
df_cleaned = df_cleaned.dropna(subset=['date'])
df_cleaned = df_cleaned.sort_values('date').reset_index(drop=True)
df_cleaned = df_cleaned.set_index('date')
df_cleaned = df_cleaned.resample('D').asfreq()
df_cleaned['pm25'] = df_cleaned['pm25'].interpolate(method='linear', limit_direction='both')
df_cleaned

,pm25
date,
2021-12-28,44.0
2021-12-29,29.0
2021-12-30,44.0
2021-12-31,49.0
2022-01-01,54.0
...,...
2025-08-07,102.0
2025-08-08,121.0
2025-08-09,107.0


In [4]:
import os
import joblib
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
warnings.filterwarnings('ignore')

# ================= CONFIG =================
SEQ_LENGTH = 60
FUTURE_DAYS = 7
BATCH_SIZE = 16
EPOCHS = 200
LR = 0.001
MODEL_NAME = "pm25_lstm_v1"

# ================= DATA PREP =================
def prepare_features(df):
    df = df.copy()
    df.index = pd.to_datetime(df.index)
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['quarter'] = df.index.quarter
    df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)
    df['hour'] = getattr(df.index, 'hour', 0)
    for lag in [1, 2, 3, 7, 14]:
        df[f'pm25_lag_{lag}'] = df.iloc[:, 0].shift(lag)
    for window in [3, 7, 14]:
        df[f'pm25_rolling_mean_{window}'] = df.iloc[:, 0].rolling(window).mean()
        df[f'pm25_rolling_std_{window}'] = df.iloc[:, 0].rolling(window).std()
    df['pm25_pct_change'] = df.iloc[:, 0].pct_change()
    df['pm25_diff'] = df.iloc[:, 0].diff()
    return df.dropna()

def create_sequences(data, seq_length, future_days):
    X, y = [], []
    for i in range(len(data) - seq_length - future_days + 1):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length:i+seq_length+future_days, 0])
    return np.array(X), np.array(y)

# ================= MODEL =================
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(256, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        BatchNormalization(),
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        BatchNormalization(),
        LSTM(64, return_sequences=False),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.1),
        Dense(64, activation='relu'),
        Dense(FUTURE_DAYS)
    ])
    optimizer = Adam(learning_rate=LR, clipnorm=1.0)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# ================= TRAIN =================
def train_model(X_train, y_train, X_val, y_val, model):
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True, min_delta=0.001),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-7, verbose=1)
    ]
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )
    return history

# ================= EVAL =================
def inverse_transform_sequences(pred_seq, scaler, num_features):
    temp = np.zeros((pred_seq.shape[0], pred_seq.shape[1], num_features))
    temp[:, :, 0] = pred_seq
    result = np.zeros_like(pred_seq)
    for i in range(pred_seq.shape[0]):
        inv = scaler.inverse_transform(temp[i])
        result[i] = inv[:, 0]
    return result

def calculate_metrics(y_true, y_pred):
    y_true_flat, y_pred_flat = y_true.flatten(), y_pred.flatten()
    mae = mean_absolute_error(y_true_flat, y_pred_flat)
    rmse = np.sqrt(mean_squared_error(y_true_flat, y_pred_flat))
    mask = y_true_flat != 0
    mape = np.mean(np.abs((y_true_flat[mask] - y_pred_flat[mask]) / y_true_flat[mask])) * 100
    r2 = 1 - np.sum((y_true_flat - y_pred_flat)**2) / np.sum((y_true_flat - np.mean(y_true_flat))**2)
    return mae, rmse, mape, r2

# ================= SAVE =================
def save_model_and_components(model, scaler, df_processed, model_name=MODEL_NAME):
    model_dir = f"{model_name}_files"
    os.makedirs(model_dir, exist_ok=True)
    model.save(os.path.join(model_dir, f"{model_name}.h5"))
    joblib.dump(scaler, os.path.join(model_dir, f"{model_name}_scaler.joblib"))
    metadata = {
        'feature_names': df_processed.columns.tolist(),
        'sequence_length': SEQ_LENGTH,
        'future_days': FUTURE_DAYS,
        'num_features': len(df_processed.columns)
    }
    joblib.dump(metadata, os.path.join(model_dir, f"{model_name}_metadata.joblib"))
    df_processed.tail(100).to_csv(os.path.join(model_dir, f"{model_name}_sample_data.csv"))
    print(f"Saved all components to {model_dir}")

# ================= MAIN =================
def main(df_cleaned):
    df = prepare_features(df_cleaned)
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df)
    X, y = create_sequences(scaled, SEQ_LENGTH, FUTURE_DAYS)
    train_size, val_size = int(len(X) * 0.7), int(len(X) * 0.15)
    X_train, y_train = X[:train_size], y[:train_size]
    X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
    X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]
    model = build_lstm_model((SEQ_LENGTH, X.shape[2]))
    history = train_model(X_train, y_train, X_val, y_val, model)
    y_pred = model.predict(X_test)
    y_pred_rescaled = inverse_transform_sequences(y_pred, scaler, X.shape[2])
    y_test_rescaled = inverse_transform_sequences(y_test, scaler, X.shape[2])
    mae, rmse, mape, r2 = calculate_metrics(y_test_rescaled, y_pred_rescaled)
    print(f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}%, R²={r2:.4f}")
    save_model_and_components(model, scaler, df)
    return model, history

if __name__ == "__main__":
    model, history = main(df_cleaned)

I0000 00:00:1755594635.907200      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1755594635.907907      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/200


I0000 00:00:1755594644.351473      99 cuda_dnn.cc:529] Loaded cuDNN version 90300


55/55 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - loss: 0.8548 - mae: 0.7429 - val_loss: 0.9634 - val_mae: 0.7684 - learning_rate: 0.0010
Epoch 2/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.7089 - mae: 0.6780 - val_loss: 1.0331 - val_mae: 0.7455 - learning_rate: 0.0010
Epoch 3/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.6576 - mae: 0.6470 - val_loss: 0.9257 - val_mae: 0.7601 - learning_rate: 0.0010
Epoch 4/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.5865 - mae: 0.6143 - val_loss: 0.8786 - val_mae: 0.7435 - learning_rate: 0.0010
Epoch 5/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.5441 - mae: 0.5896 - val_loss: 0.8696 - val_mae: 0.7117 - learning_rate: 0.0010
Epoch 6/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.5090 - mae: 0.5681 - val_loss: 0.9055 - val_mae: 0.7284 - learning_rate: 0.0010
Epoch 7/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.4915 - mae: 0.5591 - val_loss: 0.8960 - val_mae: 0.7095 - learning_rate: 0.0010
Epoch 8/200
55/55

In [6]:
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model

class PM25Predictor:
    def __init__(self, model_dir):
        self.model = load_model(
            f"{model_dir}/pm25_lstm_v1.h5",
            custom_objects={"mse": tf.keras.losses.MeanSquaredError()}
        )
        
        self.scaler = joblib.load(f"{model_dir}/pm25_lstm_v1_scaler.joblib")
        self.metadata = joblib.load(f"{model_dir}/pm25_lstm_v1_metadata.joblib")

        self.SEQ_LENGTH = self.metadata['sequence_length']
        self.FUTURE_DAYS = self.metadata['future_days']
        self.FEATURE_NAMES = self.metadata['feature_names']
        self.NUM_FEATURES = self.metadata['num_features']

    def _prepare_features(self, df):
        df = df.copy()
        df.index = pd.to_datetime(df.index)

        if 'pm25' not in df.columns:
            raise ValueError("DataFrame must contain a 'pm25' column.")

        df['day_of_week'] = df.index.dayofweek
        df['month'] = df.index.month
        df['quarter'] = df.index.quarter
        df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)
        df['hour'] = getattr(df.index, 'hour', 0)

        for lag in [1, 2, 3, 7, 14]:
            df[f'pm25_lag_{lag}'] = df['pm25'].shift(lag)

        for window in [3, 7, 14]:
            df[f'pm25_rolling_mean_{window}'] = df['pm25'].rolling(window).mean()
            df[f'pm25_rolling_std_{window}'] = df['pm25'].rolling(window).std()

        df['pm25_pct_change'] = df['pm25'].pct_change()
        df['pm25_diff'] = df['pm25'].diff()

        return df.dropna()

    def predict_next(self, df):
        """
        df: DataFrame with 'pm25' column and datetime index (at least SEQ_LENGTH days of data)
        """
        df_processed = self._prepare_features(df)
        df_processed = df_processed[self.FEATURE_NAMES]
        scaled = self.scaler.transform(df_processed)

        if scaled.shape[0] < self.SEQ_LENGTH:
            raise ValueError(f"Need at least {self.SEQ_LENGTH} days of data for prediction")

        last_seq = scaled[-self.SEQ_LENGTH:]
        X_input = np.expand_dims(last_seq, axis=0)

        y_pred_scaled = self.model.predict(X_input)

        temp = np.zeros((self.FUTURE_DAYS, self.NUM_FEATURES))
        temp[:, 0] = y_pred_scaled[0]
        y_pred_original = self.scaler.inverse_transform(temp)[:, 0]

        last_date = df.index[-1]
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=self.FUTURE_DAYS, freq='D')
        return pd.DataFrame({"date": future_dates, "predicted_pm25": y_pred_original})

# ===== Example usage =====
if __name__ == "__main__":
    predictor = PM25Predictor("pm25_lstm_v1_files")

    df_new = pd.read_csv("/kaggle/working/pm25_lstm_v1_files/pm25_lstm_v1_sample_data.csv", parse_dates=['date'], index_col='date')
    preds = predictor.predict_next(df_new)
    print(preds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 694ms/step
        date  predicted_pm25
0 2025-08-12       63.152293
1 2025-08-13       61.921403
2 2025-08-14       63.325203
3 2025-08-15       60.879182
4 2025-08-16       59.051488
5 2025-08-17       59.729240
6 2025-08-18       60.903686
